### Related equations
(arxiv: https://arxiv.org/abs/cond-mat/0207529)

The distribution of real numbers $k$ (between $-Q$ and $Q$) and $\Lambda$ (between $-B$ and $B$) are derived in this script, where $0 < Q \leq \pi$ and $0 < B \leq \infty$. The distributions are normalized by
\begin{equation}
\int_{-Q}^{Q} \rho(k)dk = N/N_a \hspace{0.3cm} (\text{filling}) , \hspace{0.5cm} \int_{-B}^{B} \sigma(\Lambda)d\Lambda = M/N_a \hspace{0.3cm} (\text{density of down spin})
\end{equation}

The equations to solve $\rho(k)$ and $\sigma(\Lambda)$ are
\begin{equation}
\rho(k) = \frac{1}{2\pi}+\cos k\int_{-B}^{B} K(\sin k - \Lambda)\sigma(\Lambda)d\Lambda
\end{equation}
\begin{equation}
\sigma(\Lambda) = \int_{-Q}^{Q} K(\sin k -\Lambda)\rho(k)dk - \int_{-B}^{B} K^2(\Lambda-\Lambda')\sigma(\Lambda')d\Lambda'
\end{equation}
where
\begin{equation}
K(\Lambda - \Lambda') = \frac{1}{2\pi}\left[\frac{8U}{U^2 + 16(\Lambda-\Lambda')^2}\right]
\end{equation}
\begin{equation}
K^2(\Lambda - \Lambda') = \frac{1}{2\pi}\left[\frac{4U}{U^2 + 4(\Lambda-\Lambda')^2}\right]
\end{equation}
Half-filling and $S_z = 0$
$$Q = \pi, \hspace{0.5cm} B = \infty$$ 

Energy is
$$E(M, N) = -2N_a \int_{-Q}^{Q} \rho(k)\cos kdk$$

### Implementation
Rewrite integral equations of $\rho(k)$ and $\sigma(\Lambda)$ with descrete formula (matrix representation).

\begin{equation}
rho_k = C^0_k + C_k * K_{kl}D^l_{ll}\sigma_l
\end{equation}

\begin{equation}
\sigma_l = K_{kl}D^k_{kk}\rho_k - K^2_{ll'}D^l_{l'l'}\sigma_l
\end{equation}

where $\rho(k) \rightarrow \rho_k$, $\sigma(\Lambda) \rightarrow \sigma_l$, $\frac{1}{2\pi} \rightarrow C^0_k$, $\cos k\rightarrow C_k$, $K(\sin k-\Lambda)\rightarrow K_{kl}$, and $K^2(\Lambda-\Lambda')\rightarrow K^2_{ll'}$.

$D^k$ and $D^l$ are identity matrices multiplied by $dk$ and $dl$ respectively.

Let  If we concatenate $P_k$ and $S_l$, we get
\begin{equation}
\begin{pmatrix}I & -CKD^l \\ -K^TD^k & I+D^l(K^2)^T\end{pmatrix}
\begin{pmatrix}P \\ S\end{pmatrix} 
= \begin{pmatrix}C^0 \\ 0\end{pmatrix}
\end{equation}

In [128]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [171]:
class BatheAnsatzGS(object):
    def __init__(self,U, filling=1, ngrid=20, tol=5e-2, maxiter=50):
        # Assume that Sz = 0
        self.U = U
        self.filling = filling
        # initialization 
        self.Q = np.pi 
        self.B = 0 
        self.Qmax = np.pi
        self.Bmax = 20
        self.ngrid = ngrid
        self.kgrid = np.zeros(ngrid)
        self.lgrid = np.zeros(ngrid)
        self.rhok = np.zeros(ngrid)
        self.sigl = np.zeros(ngrid)
        self.tol = tol
        self.maxiter = maxiter
        
    def K_func(self, k, l):
        U = self.U
        return 8.*U/(2.*np.pi*(U**2 + 16*(np.sin(k)-l)**2))
    def K2_func(self, x, xp):
        U = self.U
        return 4.*U/(2.*np.pi*(U**2 + 4*(x-xp)**2))
    def energy_per_site(self):
        dk = self.kgrid[1]-self.kgrid[0]
        return -2.*np.dot(self.rhok, np.cos(self.kgrid))*dk
    def get_filling(self):
        dk = self.kgrid[1]-self.kgrid[0]
        return np.sum(self.rhok)*dk
    def get_spindown(self):
        dl = self.lgrid[1]-self.lgrid[0]
        return np.sum(self.sigl)*dl
    
    def solve_density(self):
        ngrid = self.ngrid
        dk = self.kgrid[1]-self.kgrid[0]
        dl = self.lgrid[1]-self.lgrid[0]
        Cmat = np.zeros((ngrid*2,)*2)
        Kmat = np.zeros((ngrid,ngrid))
        K2mat = np.zeros((ngrid,ngrid))
        for k in range(ngrid):
            for l in range(ngrid):
                Kmat[k,l] = self.K_func(self.kgrid[k], self.lgrid[l])
                K2mat[k,l] = self.K2_func(self.lgrid[k], self.lgrid[l])
        Cmat[:ngrid, :ngrid] = np.eye(ngrid)
        Cmat[:ngrid, ngrid:] = -np.dot(np.dot(np.diag(np.cos(self.kgrid)), Kmat), np.eye(ngrid)*dl)
        Cmat[ngrid:, :ngrid] = -np.dot(np.eye(ngrid)*dk, Kmat.T)
        Cmat[ngrid:, ngrid:] = np.eye(ngrid)+np.dot(np.eye(ngrid)*dl,K2mat.T)
        const_vec = np.zeros(ngrid*2)
        const_vec[:ngrid] = np.ones(ngrid)/(2.*np.pi)
        results = np.linalg.solve(Cmat, const_vec)
        self.rhok = results[:ngrid]
        self.sigl = results[ngrid:]
        
    def kernel(self):
        Qmin = 0.01
        Bmin = 0
        nstep = self.maxiter
        self.Q = Qmin
        self.B = self.Bmax
        dQ = (self.Qmax - Qmin)/nstep
        dB = (self.Bmax - Bmin)/nstep
        for i in range(self.maxiter):
            self.kgrid = np.linspace(-self.Q, self.Q, self.ngrid)
            self.lgrid = np.linspace(-self.B, self.B, self.ngrid)
            self.solve_density()
            tmp_fill = self.get_filling()
            #print "filling:", tmp_fill
            tmp_spind = self.get_spindown()
            diff = abs(tmp_fill-self.filling)
            #print "diff: ", diff
            if(diff < self.tol):
                print "The calculation converged with %d loops!\n"%(i+1)
                break
            if (i==self.maxiter-1):
                print "The calculation did not converge after %d loops!\n"%(self.maxiter)
            self.Q += dQ
            #self.B += dB
        energy = self.energy_per_site()
        print "Final filling: %0.4f\n"%tmp_fill
        print "Final density of spin down: %0.4f\n"%self.get_spindown()
        print "Energy per site at U = %0.2f: %0.6f\n"%(U, energy)
        return energy

In [176]:
U = 1
ba = BatheAnsatzGS(U, filling=1, ngrid=50, tol=1e-3, maxiter=100)

In [177]:
ba.kernel()

The calculation converged with 99 loops!

Final filling: 1.0000

Final density of spin down: 0.4540

Energy per site at U = 1.00: -0.949685



-0.94968504177485857